# 반복형, 반복자, 제너레이터

- 데이터를 처리할 때 반복은 기본
- 메모리에 다 들어가지 않는 데이터셋 검색할 때 항목들을 **느긋하게** 가져와야 함
    - 즉, 한번에 한번씩 필요할 때 가져와야 함
    - 반복자 패턴이 하는 일

**Note**
- 모든 제너레이터는 반복자
    - 제너레이터가 반복자 인터페이스를 완전히 구현하고 있기 때문
- 반복자는 [디자인 패턴] 에서 정의한 대로 컬렉션에서 항목을 가지고 옴
- 제너레이터는 '마술처럼' 항목 생성 가능
- 파이썬 커뮤니티에서는 **반복자** 와 **제너레이터**를 거의 동일시 

파이썬의 **컬렉션**은 모두 **반복형**, 다음과 같은 연산 지원하기 위해 내부적으로 반복자 사용
- for 루프
- 컬렉션형 생성과 확장
- 텍스트 파일을 한 줄씩 반복
- 지능형 리스트/딕셔너리/집합
- 튜플 언패킹
- 함수 호출 시 * 를 이용해서 실제 매개변수를 언패킹

## 14.1 Sentence 버전 #1 : 단어 시퀀스

#### 예제 14-1 sentence.py : 단어 시퀀스로서의 Sentence 클래스

In [5]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 정규 표현식에서 매칭되는 중복되지 않는 문자열의 리스트를 반환
    
    def __getitem__(self, index):
        return self.words[index] # 주어진 index에 해당하는 단어 반환
    
    def __len__(self): # 시퀀스 프로토콜에 따르려면 필요, 반복형 객체에 이 메서드가 필요한 것은 아님
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 매우 큰 데이터 구조체를 표현하는 문자열 축약해서 생성, 문자열을 30자로 제한

#### 14-2 Sentence 객체의 반복 테스트

In [6]:
s = Sentence('"The time has come," the  Walrus said,') # 문자열을 이용하여 Sentence 객체 생성
s # 출력한 메세지는 reprlib.repr()이 생성

Sentence('"The time ha... Walrus said,')

In [8]:
for word in s: # Sentence 객체는 반복할 수 있음
    print(word)

The
time
has
come
the
Walrus
said


In [10]:
list(s) # 반복할 수 있으므로 Sentece 객체는 리스트 혹은 다른 반복형을 생성하기 위한 입력으로 사용 가능

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

<hr>

### 14.1.1 Sequemce가 반복 가능한 이유 : iter() 함수

- 파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 iter(x)를 자동으로 호출
- iter() 내장함수는 다음 과정 수행
    1. 객체가 \__iter\__() 메서드를 구현하는지 확인, 이 메서드를 호출해서 반복자 가지고 옴
    2. \__iter\__()  메서드가 구현되어 있지 않지만 \__getitem\__() 이 구현되어 있다면, 파이썬은 인덱스 0에서 시작해서 항목을 순서대로 가져오는 반복자 생성
    3. 이 과정이 모두 실패하면 파이썬은 **TypeError ; 'C' object is not iterable** 발생

- 그렇기 때문에 모든 파이썬 시퀀스는 반복 가능
- 사실 표준 시퀀스는 \__iter\__() 메서드도 구현하고 있으므로 이 메서드도 구현

<br>


In [8]:
class Foo:
    def __iter__(self):
        pass
    

from collections import abc

print(issubclass(Foo, abc.Iterable))


f = Foo()
print(isinstance(f, abc.Iterable))

########################################################

print(issubclass(Sentence, abc.Iterable))


s = Sentence("e")
print(isinstance(s, abc.Iterable))

True
True
False
False


## 14.2 반복형과 반복자

**반복형**
- iter() 내장 함수가 반복자를 가져올 수 있는 모든 객체와 반복자를 반환하는 \__iter\__() 메서드를 구현하는 객체는 반복형
- 0에서 시작하는 인덱스를 받는 \__getitem\__() 메서드를 구현하는 객체인 시퀀스도 마찬가지

In [10]:
s = "ABC" # 'ABC' 문자열 반복형, 반복자 보이지 않지만 내부 어딘가 존재

it = iter(s) # 반복형에서 반복자 it을 생성

while True:
    try:
        print(next(it)) # 반복자에서 next를 계속 호출해서 다음 항목을 가져옴 
    except StopIteration: # 더 이상 항목이 없으면 예외 발생
        del it # it에 대한 참조 해제하여 반복자 객체 제겨
        break

A
B
C


**\__next\__()**
- 다음에 사용할 항목 반환
- 더 이상 항목이 남아있지 않으면 StopIteration 발생

<br>

**\__self\__()**
- self 반환
- for 루프 등 반복형이 필요한 곳에 반복자 사용할 수 있게 만듦

<img src="./images/3.png">

- Iterable과 Iterator ABC
- Iterable의 구상 서브 클래스의 \__iter\__() 메서드는 Iterator 객체를 생성하고 반환해야 함
- Iterator의 구상 서브 클래스는 \__next\__() 메서드를 구현해야 함
- Iterator.\__iter\__() 메서드는 self를 반환

#### 예제 14-3 abc.Iterator 클래스의 일부

In [14]:
class Iterator(Iterable):
    
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        '''반복자에서 다음 항목을 반환, 항목이 소진되면 StopIteeration 예외 발생'''
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
            
        return NotImplemented

NameError: name 'Iterable' is not defined

<hr>

In [19]:
s3 = Sentence("Pig and Pepper") # 세 단어로 구성된 Sentece 객체 생성
it = iter(s3) # s3에서 반복자 가지고 옴
print(it)

print("-" * 30)

print(next(it)) # 다음단어 가지고 옴
print(next(it)) # 다음단어 가지고 옴
print(next(it)) # 다음단어 가지고 옴
print(next(it)) # 다음단어 가지고 옴


------------------------------
Pig
and
Pepper


StopIteration: 

In [21]:
print(list(it)) # 소진 된 후에는 반복자 필요 없음
print(list(iter(s3))) # Sentence를 다시 반복하려면 생성자 새로 만들어야 함

[]
['Pig', 'and', 'Pepper']


<hr>

- 반복자가 필수적으로 구현해야 하는 메서드는 \__next\__()와 \__iter\__()